## HW 3: Fairness and Bias Interventions

## Download the dataset

1. Go to the [Adult Dataset webpage](https://archive.ics.uci.edu/dataset/2/adult).
2. Download and unzip the file in the same directory as this notebook.

In [30]:
# TODO: download
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import math

import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix
from tqdm import tqdm

In [31]:
## DATA LOADING ##
header = ["age",
          "workclass",
          "fnlwgt",
          "education",
          "education-num",
          "marital-status",
          "occupation",
          "relationship",
          "race",
          "sex",
          "capital-gain",
          "capital-loss",
          "hours-per-week",
          "native-country"]

values = {"workclass": ["Private", "Self-emp-not-inc", "Self-emp-inc", "Federal-gov", "Local-gov", "State-gov", "Without-pay", "Never-worked"],
          "education": ["Bachelors", "Some-college", "11th", "HS-grad", "Prof-school", "Assoc-acdm", "Assoc-voc", "9th", "7th-8th", "12th", "Masters", "1st-4th", "10th", "Doctorate", "5th-6th", "Preschool"],
          "marital-status": ["Married-civ-spouse", "Divorced", "Never-married", "Separated", "Widowed", "Married-spouse-absent", "Married-AF-spouse"],
          "occupation": ["Tech-support", "Craft-repair", "Other-service", "Sales", "Exec-managerial", "Prof-specialty", "Handlers-cleaners", "Machine-op-inspct", "Adm-clerical", "Farming-fishing", "Transport-moving", "Priv-house-serv", "Protective-serv", "Armed-Forces"],
          "relationship": ["Wife", "Own-child", "Husband", "Not-in-family", "Other-relative", "Unmarried"],
          "race": ["White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"],
          "sex": ["Female", "Male"],
          "native-country": ["United-States", "Cambodia", "England", "Puerto-Rico", "Canada", "Germany", "Outlying-US(Guam-USVI-etc)", "India", "Japan", "Greece", "South", "China", "Cuba", "Iran", "Honduras", "Philippines", "Italy", "Poland", "Jamaica", "Vietnam", "Mexico", "Portugal", "Ireland", "France", "Dominican-Republic", "Laos", "Ecuador", "Taiwan", "Haiti", "Columbia", "Hungary", "Guatemala", "Nicaragua", "Scotland", "Thailand", "Yugoslavia", "El-Salvador", "Trinadad&Tobago", "Peru", "Hong", "Holand-Netherlands"]
         }

In [32]:
def feat(d):
    f = [1]
    for h in header:
        if h in values:
            onehot = [0]*len(values[h])
            try:
                onehot[values[h].index(d[h])] = 1 # not efficient! Should make an index
            except Exception as e:
                # Missing value
                pass
            f += onehot
        else: # continuous
            try:
                f.append(float(d[h]))
            except Exception as e:
                # Missing value
                f.append(0) # Replacing with zero probably not perfect!
    return f

In [33]:
dataset = []
labels = []
a = open("adult.data", "r")
for l in a:
    if len(l) <= 1: break # Last line of the dataset is empty
    l = l.split(", ") # Could use a csv library but probably no need to here
    dataset.append(dict(zip(header, l)))
    labels.append(l[-1].strip()) # Last entry in each row is the label

X = [feat(d) for d in dataset]
y = [inc == ">50K" for inc in labels]

X_train, X_test, y_train, y_test, d_train, d_test = train_test_split(X, y, dataset, test_size=0.2, random_state=42)

In [34]:
type(X_train), type(y_train), type(d_train), type(X_test), type(y_test), type(d_test)

(list, list, list, list, list, list)

In [35]:
print(len(X_train[0]))

106


In [36]:
len(X_train), len(y_train), len(d_train), len(X_test), len(y_test), len(d_test)

(26048, 26048, 26048, 6513, 6513, 6513)

In [37]:
d_train[0], y_train[0], X_train[0]

({'age': '33',
  'workclass': 'Local-gov',
  'fnlwgt': '198183',
  'education': 'Bachelors',
  'education-num': '13',
  'marital-status': 'Never-married',
  'occupation': 'Prof-specialty',
  'relationship': 'Not-in-family',
  'race': 'White',
  'sex': 'Female',
  'capital-gain': '0',
  'capital-loss': '0',
  'hours-per-week': '50',
  'native-country': 'United-States'},
 True,
 [1,
  33.0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  198183.0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  13.0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0.0,
  0.0,
  50.0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [38]:
answers = {}

## 3.1

#### (1 point)

Implement a logistic regression classification pipeline using an `80/20` test split. Use a regularization value of $C = 1$.

Treat “sex” as the “sensitive attribute” i.e., $z=1$ for females and $z=0$ for others.

**Report:** The discrimination in the dataset (see "pre-processing" module).

In [39]:
# [YOUR CODE HERE]

def discrimination_score(datapoints, labels):
    z0y1 = 0
    z0y0 = 0
    z1y0 = 0
    z1y1 = 0
    for datapoint, label in zip(datapoints, labels):
        if datapoint["sex"] == "Female" and label == True:
            z1y1 += 1
        elif datapoint["sex"] == "Female" and label == False:
            z1y0 += 1
        elif datapoint["sex"] != "Female" and label == True:
            z0y1 += 1
        elif datapoint["sex"] != "Female" and label == False:
            z0y0 += 1
    return z0y1 / (z0y0 + z0y1) - z1y1 / (z1y0 + z1y1)

dataset_discrimination = discrimination_score(d_train, y_train)
print(f"Dataset discrimination: {dataset_discrimination:.6f}")

Dataset discrimination: 0.196497


In [40]:
answers["Q1"] = dataset_discrimination

## 3.2

#### (1 point)

**Report:** The discrimination of the classifier.

In [41]:
# [YOUR CODE HERE]

model_q2 = LogisticRegression(C=1)
model_q2.fit(X_train, y_train)
preds_train_q2 = model_q2.predict(X_train)
classifier_discrimination_q2 = discrimination_score(d_train, preds_train_q2)

print(f"Classifier discrimination (Q2): {classifier_discrimination_q2:.6f}")

Classifier discrimination (Q2): 0.042491


In [42]:
answers["Q2"] = classifier_discrimination_q2

## 3.3
#### (1 point)

Implement a "massaging" approach that improves the discrimination score by at least 3\%.


**Report:** The new discrimination score.

In [43]:
promote_candidates = [i for i, (y, d) in enumerate(zip(y_train, d_train)) if y == False and d["sex"] == "Female"]
print(len(promote_candidates))
demote_candidates = [i for i, (y, d) in enumerate(zip(y_train, d_train)) if y == True and d["sex"] != "Female"]
print(len(demote_candidates))

count0 = 0
count1 = 0
for i in range(len(y_train)):
    if y_train[i] == False:
        count0 += 1
    else:
        count1 += 1
M = 2 * int(classifier_discrimination_q2 * count0 * count1 / (count0 + count1))
print(M)

y_train_fixed_q3 = y_train.copy()
for i in promote_candidates[:M]:
    y_train_fixed_q3[i] = 1
for i in demote_candidates[:M]:
    y_train_fixed_q3[i] = 0

model_q3 = LogisticRegression(C=1)
model_q3.fit(X_train, y_train_fixed_q3)
preds_train_q3 = model_q3.predict(X_train)
classifier_discrimination_q3 = discrimination_score(d_train, preds_train_q3)

print(f"Classifier discrimination (Q3):\t {classifier_discrimination_q3:.6f}")
print(f"Classifier relative improvement (Q3):\t {100*(classifier_discrimination_q2 - classifier_discrimination_q3) / classifier_discrimination_q2:.6f}%")

7699
5324
404
Classifier discrimination (Q3):	 0.040612
Classifier relative improvement (Q3):	 4.421157%


In [44]:
answers["Q3"] = classifier_discrimination_q3

## 3.4

#### (2 points)

Implement a "reweighting" approach that improves the discrimination score by at least 3%; report the new discrimination score.

**Report:** The new discrimination score.

In [45]:
y_train_np = np.array(y_train)
d_train_np = np.array([1 if d["sex"] == "Female" else 0 for d in d_train])

w_pos_female = np.mean(y_train_np == 1) / np.mean((y_train_np == 1) & (d_train_np == 1))
w_neg_female = (1 - np.mean(y_train_np == 1)) / np.mean((y_train_np == 0) & (d_train_np == 1))

w_pos_male = np.mean(y_train_np == 1) / np.mean((y_train_np == 1) & (d_train_np == 0))
w_neg_male = (1 - np.mean(y_train_np == 1)) / np.mean((y_train_np == 0) & (d_train_np == 0))

weights = np.where((y_train_np == 1) & (d_train_np == 1), w_pos_female, np.where((y_train_np == 0) & (d_train_np == 1), w_neg_female, np.where((y_train_np == 1) & (d_train_np == 0), w_pos_male, w_neg_male)))
print(weights)

model_q4 = LogisticRegression(C=1)
model_q4.fit(X_train, y_train, sample_weight=weights)
preds_train_q4 = model_q4.predict(X_train)
classifier_discrimination_q4 = discrimination_score(d_train, preds_train_q4)

print(f"Classifier discrimination (Q4):\t {classifier_discrimination_q4:.6f}")
print(f"Classifier relative improvement (Q4):\t {100*(classifier_discrimination_q2 - classifier_discrimination_q4) / classifier_discrimination_q2:.6f}%")

[6.62790698 1.17768595 1.6373872  ... 2.56890505 1.6373872  1.6373872 ]
Classifier discrimination (Q4):	 0.038644
Classifier relative improvement (Q4):	 9.053541%


In [46]:
answers["Q4"] = classifier_discrimination_q4

## 3.5

#### (2 points)

Implement a "post processing" (affirmative action) policy. Lowering per-group thresholds will increase both the (per-group) FPR and the (per-group) TPR. For whichever group has the lower TPR, lower the threshold until the TPR for both groups is (as close as possible to) equal. Report the rates (TPR_0, TPR_1, FPR_0, and FPR_1) for both groups.

**Report:** The TPR and FPR rates for both groups as a list: `[TPR_0, TPR_1, FPR_0, FPR_1]`.


In [47]:
pred_probs = model_q2.predict_proba(X_train)[:, 1]

male_indices = [i for i, d in enumerate(d_train) if d["sex"] != "Female"]
female_indices = [i for i, d in enumerate(d_train) if d["sex"] == "Female"]

def compute_rates(y_true, y_scores, threshold):
    y_scores = np.array(y_scores)
    preds = (y_scores >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    return tpr, fpr

male_thresh = 0.5

tpr_male, fpr_male = compute_rates([y_train[i] for i in male_indices], [pred_probs[i] for i in male_indices], 0.5)
print(tpr_male, fpr_male)
tpr_female, fpr_female = compute_rates([y_train[i] for i in female_indices], [pred_probs[i] for i in female_indices], 0.5)
print(tpr_female, fpr_female)

min_diff = 1000
thresholds = sorted(set(pred_probs[i] for i in male_indices), reverse=True)
for thresh in tqdm(thresholds, total=len(thresholds)):
    new_tpr, _ = compute_rates([y_train[i] for i in male_indices], [pred_probs[i] for i in male_indices], thresh)
    if abs(new_tpr - tpr_female) < min_diff and thresh < 0.5:
        male_thresh = thresh
        min_diff = abs(new_tpr - tpr_female)
print(male_thresh)

tpr_male, fpr_male = compute_rates([y_train[i] for i in male_indices], [pred_probs[i] for i in male_indices], male_thresh)
print(tpr_male, fpr_male)

ans_q5 = [float(tpr_male), float(tpr_female), float(fpr_male), float(fpr_female)]

print(ans_q5)

0.25187828700225395 0.03352926566768772
0.2928118393234672 0.028964800623457593


100%|██████████| 17360/17360 [02:48<00:00, 102.80it/s]

0.3860845631546153
0.29282494365138995 0.06912823909264013
[0.29282494365138995, 0.2928118393234672, 0.06912823909264013, 0.028964800623457593]


In [48]:
answers["Q5"] = ans_q5

## 3.6

#### (1 point)

Modify the solution from Q5 to exclude the sensitive attribute ($z$) from the classifier’s feature vector. Implement the same strategy as in Q5.

**Report:** The TPR and FPR rates for both groups as a list: `[TPR_0, TPR_1, FPR_0, FPR_1]`.


In [49]:
X_train =np.delete(X_train, [60, 61], axis=1)
model_q6 = LogisticRegression(C=1)
model_q6.fit(X_train, y_train)

LogisticRegression(C=1)

In [50]:
pred_probs = model_q6.predict_proba(X_train)[:, 1]

male_indices = [i for i, d in enumerate(d_train) if d["sex"] != "Female"]
female_indices = [i for i, d in enumerate(d_train) if d["sex"] == "Female"]

def compute_rates(y_true, y_scores, threshold):
    y_scores = np.array(y_scores)
    preds = (y_scores >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    return tpr, fpr

tpr_male, fpr_male = compute_rates([y_train[i] for i in male_indices], [pred_probs[i] for i in male_indices], 0.5)
print(tpr_male, fpr_male)
tpr_female, fpr_female = compute_rates([y_train[i] for i in female_indices], [pred_probs[i] for i in female_indices], 0.5)
print(tpr_female, fpr_female)

male_thresh = 0.5

min_diff = 1000
thresholds = sorted(set(pred_probs[i] for i in male_indices), reverse=True)
# thresholds = np.linspace(0, 0.5, 10000)
for thresh in tqdm(thresholds, total=len(thresholds)):
    new_tpr, _ = compute_rates([y_train[i] for i in male_indices], [pred_probs[i] for i in male_indices], thresh)
    # if abs(new_tpr - tpr_female) < min_diff and thresh < 0.5:
    if tpr_female - new_tpr < min_diff and tpr_female - new_tpr > 0 and thresh < 0.5:
        male_thresh = thresh
        min_diff = abs(new_tpr - tpr_female)
print(male_thresh)

tpr_male, fpr_male = compute_rates([y_train[i] for i in male_indices], [pred_probs[i] for i in male_indices], male_thresh)
print(tpr_male, fpr_male)

female_thresh_q6 = 0.5
ans_q6 = [float(tpr_male), float(tpr_female), float(fpr_male), float(fpr_female)]

print(ans_q6)

0.24436513899323817 0.02930706184286779
0.2917547568710359 0.026886608650474087


100%|██████████| 17341/17341 [02:36<00:00, 110.96it/s]

0.3756004837954219
0.29169797145003756 0.06440930540607666
[0.29169797145003756, 0.2917547568710359, 0.06440930540607666, 0.026886608650474087]


In [51]:
answers["Q6"] = ans_q6

## 3.7

#### (1 point)

Again modifying the solution from Q5, train two separate classifiers, one for $z=0$ and one for $z=1$. Implement the same strategy as in Q5.

**Report:** The TPR and FPR rates for both groups as a list: `[TPR_0, TPR_1, FPR_0, FPR_1]`.

In [52]:
## DATA LOADING ##
header = ["age",
          "workclass",
          "fnlwgt",
          "education",
          "education-num",
          "marital-status",
          "occupation",
          "relationship",
          "race",
          "sex",
          "capital-gain",
          "capital-loss",
          "hours-per-week",
          "native-country"]

values = {"workclass": ["Private", "Self-emp-not-inc", "Self-emp-inc", "Federal-gov", "Local-gov", "State-gov", "Without-pay", "Never-worked"],
          "education": ["Bachelors", "Some-college", "11th", "HS-grad", "Prof-school", "Assoc-acdm", "Assoc-voc", "9th", "7th-8th", "12th", "Masters", "1st-4th", "10th", "Doctorate", "5th-6th", "Preschool"],
          "marital-status": ["Married-civ-spouse", "Divorced", "Never-married", "Separated", "Widowed", "Married-spouse-absent", "Married-AF-spouse"],
          "occupation": ["Tech-support", "Craft-repair", "Other-service", "Sales", "Exec-managerial", "Prof-specialty", "Handlers-cleaners", "Machine-op-inspct", "Adm-clerical", "Farming-fishing", "Transport-moving", "Priv-house-serv", "Protective-serv", "Armed-Forces"],
          "relationship": ["Wife", "Own-child", "Husband", "Not-in-family", "Other-relative", "Unmarried"],
          "race": ["White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"],
          "sex": ["Female", "Male"],
          "native-country": ["United-States", "Cambodia", "England", "Puerto-Rico", "Canada", "Germany", "Outlying-US(Guam-USVI-etc)", "India", "Japan", "Greece", "South", "China", "Cuba", "Iran", "Honduras", "Philippines", "Italy", "Poland", "Jamaica", "Vietnam", "Mexico", "Portugal", "Ireland", "France", "Dominican-Republic", "Laos", "Ecuador", "Taiwan", "Haiti", "Columbia", "Hungary", "Guatemala", "Nicaragua", "Scotland", "Thailand", "Yugoslavia", "El-Salvador", "Trinadad&Tobago", "Peru", "Hong", "Holand-Netherlands"]
         }

In [53]:
def feat(d):
    f = [1]
    for h in header:
        if h in values:
            onehot = [0]*len(values[h])
            try:
                onehot[values[h].index(d[h])] = 1 # not efficient! Should make an index
            except Exception as e:
                # Missing value
                pass
            f += onehot
        else: # continuous
            try:
                f.append(float(d[h]))
            except Exception as e:
                # Missing value
                f.append(0) # Replacing with zero probably not perfect!
    return f

In [54]:
dataset = []
labels = []
a = open("adult.data", "r")
for l in a:
    if len(l) <= 1: break # Last line of the dataset is empty
    l = l.split(", ") # Could use a csv library but probably no need to here
    dataset.append(dict(zip(header, l)))
    labels.append(l[-1].strip()) # Last entry in each row is the label

X = [feat(d) for d in dataset]
y = [inc == ">50K" for inc in labels]

X_train, X_test, y_train, y_test, d_train, d_test = train_test_split(X, y, dataset, test_size=0.2, random_state=42)

In [55]:
male_indices = [i for i, d in enumerate(d_train) if d["sex"] != "Female"]
female_indices = [i for i, d in enumerate(d_train) if d["sex"] == "Female"]

X_train_male = [X_train[i] for i in male_indices]
X_train_female = [X_train[i] for i in female_indices]
y_train_male  = [y_train[i] for i in male_indices]
y_train_female = [y_train[i] for i in female_indices]

model_male_q7 = LogisticRegression(C=1)
model_female_q7 = LogisticRegression(C=1)

model_male_q7.fit(X_train_male, y_train_male)
model_female_q7.fit(X_train_female, y_train_female)

pred_probs_male = model_male_q7.predict_proba(X_train_male)[:, 1]
pred_probs_female = model_female_q7.predict_proba(X_train_female)[:, 1]


def compute_rates(y_true, y_scores, threshold):
    y_scores = np.array(y_scores)
    preds = (y_scores >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    return tpr, fpr

tpr_male, fpr_male = compute_rates(y_train_male, pred_probs_male, 0.5)
print(tpr_male, fpr_male)
tpr_female, fpr_female = compute_rates(y_train_female, pred_probs_female, 0.5)
print(tpr_female, fpr_female)

male_thresh = 0.5

female_thresh_q7 = 0.5
min_diff = 1000
thresholds = sorted(set(pred_probs_female), reverse=True)
# thresholds = np.linspace(0, 0.5, 10000)
for thresh in tqdm(thresholds, total=len(thresholds)):
    new_tpr, _ = compute_rates(y_train_female, pred_probs_female, thresh)
    if abs(new_tpr - tpr_male) < min_diff and thresh < 0.5:
        female_thresh_q7 = thresh
        min_diff = abs(new_tpr - tpr_male)
print(female_thresh_q7)

tpr_female, fpr_female = compute_rates(y_train_female, pred_probs_female, female_thresh_q7)
print(tpr_female, fpr_female)

ans_q7 = [float(tpr_male), float(tpr_female), float(fpr_male), float(fpr_female)]
ans_q7

0.28568745304282495 0.05223942379336038
0.2357293868921776 0.01052084686322899


100%|██████████| 8620/8620 [00:27<00:00, 312.68it/s]

0.3166615774052531
0.2854122621564482 0.0314326535913755


[0.28568745304282495,
 0.2854122621564482,
 0.05223942379336038,
 0.0314326535913755]

In [56]:
answers["Q7"] = ans_q7

In [57]:
answers

{'Q1': 0.1964968512405887,
 'Q2': 0.042490629794089275,
 'Q3': 0.04061205218900353,
 'Q4': 0.03864372320083223,
 'Q5': [0.29282494365138995,
  0.2928118393234672,
  0.06912823909264013,
  0.028964800623457593],
 'Q6': [0.29169797145003756,
  0.2917547568710359,
  0.06440930540607666,
  0.026886608650474087],
 'Q7': [0.28568745304282495,
  0.2854122621564482,
  0.05223942379336038,
  0.0314326535913755]}